In [18]:
import requests
import re
from requests_html import HTML
from selenium import webdriver
import time
from selenium.webdriver.chrome.options import Options # for headless mode


In [19]:
options = Options()
options.add_argument("--headless") # Runs Chrome in headless mode.
driver = webdriver.Chrome(options=options)


Exception managing chrome: error sending request for url (https://googlechromelabs.github.io/chrome-for-testing/known-good-versions-with-downloads.json): error trying to connect: dns error: No such host is known. (os error 11001)
The chromedriver version (120.0.6099.109) detected in PATH at C:\chromedriver\chromedriver.exe might not be compatible with the detected chrome version (121.0.6167.161); currently, chromedriver  is recommended for chrome 121.*, so it is advised to delete the driver in PATH and retry
Error sending stats to Plausible: error sending request for url (https://plausible.io/api/event): error trying to connect: dns error: No such host is known. (os error 11001)


In [20]:
categories = [
    "https://www.amazon.in/gp/bestsellers/boost/ref=zg_bs_nav_boost_0",
    "https://www.amazon.in/gp/bestsellers/electronics/ref=zg_bs_nav_electronics_0",
    "https://www.amazon.in/gp/bestsellers/electronics/1389335031/ref=zg_bs_nav_electronics_1",]


In [21]:
first_url = categories[0]

In [22]:
driver.get(first_url)

In [23]:
body_el = driver.find_element(by="tag name", value="body")
html_str = body_el.get_attribute("innerHTML")

In [24]:
html_obj = HTML(html=html_str)

In [25]:
new_links = [x for x in html_obj.links if x.startswith("/")]
print(new_links)
new_links=[x for x in new_links if "product-reviews/" not in x]

['/Sirona-Approved-Reusable-Menstrual-Silicone/dp/B07548J9WC/ref=zg_bs_g_boost_d_sccl_3/261-1198201-1142810?psc=1', '/product-reviews/B07J6GBG8H/ref=zg_bs_g_boost_d_sccl_19_cr/261-1198201-1142810', '/Hykes-Compression-Support-Running-Cycling/dp/B07J6GBG8H/ref=zg_bs_g_boost_d_sccl_19/261-1198201-1142810?psc=1', '/prime?ref_=nav_cs_primelink_nonmember', '/Sirona-Disposable-Underarm-Peel-off-Unpleasant/dp/B01BKBOGDY/ref=zg_bs_g_boost_d_sccl_8/261-1198201-1142810?psc=1', '/now?ref=footer_amznow', '/FIXDERMA-Sunscreen-30-Protection-Resistant/dp/B089RJ6Z3K/ref=zg_bs_g_boost_d_sccl_5/261-1198201-1142810?psc=1', '/gp/browse.html?node=6648217031&ref_=nav_cs_fashion', '/product-reviews/B08SQ6S5HF/ref=zg_bs_g_boost_d_sccl_27_cr/261-1198201-1142810', '/gp/bestsellers/boost/10894277031/ref=zg_bs_nav_boost_1', '/gp/help/customer/display.html?nodeId=200534380&ref_=footer_privacy', '/hz/mycd/myx?pageType=content&ref_=nav_AccountFlyout_myk', '/product-reviews/B07548J9WC/ref=zg_bs_g_boost_d_sccl_3_cr/26

In [26]:
class_options=[".a-price-whole", ".a-offscreen", ]


def scrape_product_page(

    url,
    title_lookup="#productTitle",

):

    driver.get(url)

    time.sleep(1.2)

    body_el = driver.find_element(by="tag name", value="body")  
    html_str = body_el.get_attribute("innerHTML")

    html_obj = HTML(html=html_str)
   
    product_title = html_obj.find(title_lookup, first=True).text
    for price_lookup in class_options:
        product_price = html_obj.find(price_lookup, first=True).text
    return product_title, product_price

In [27]:
my_regex_pattern = r"https://www.amazon.in/(?P<slug>[\w-]+)/dp/(?P<product_id>[\w]+)/"
my_url = "https://www.amazon.in/SOFTSPUN-Microfiber-Cleaning-Cloth-340GSM/dp/B07GBZ1KV8/"

In [28]:
regex=re.compile(my_regex_pattern)

In [29]:
my_match = regex.match(my_url)
print(my_match)
my_match["product_id"]
my_match["slug"]

<re.Match object; span=(0, 78), match='https://www.amazon.in/SOFTSPUN-Microfiber-Cleanin>


'SOFTSPUN-Microfiber-Cleaning-Cloth-340GSM'

In [30]:
regex_options=[
    "https://www.amazon.in/(?P<slug>[\w-]+)/dp/(?P<product_id>[\w]+)/",
    "https://www.amazon.in/dp/(?P<product_id>[\w]+)/",
    "https://www.amazon.in/gp/product/(?P<product_id>[\w]+)/",
]

def extract_product_id_from_url(url):
    product_id=None
    for regex_str in regex_options:
        regex=re.compile(regex_str)
        my_match = regex.match(url)
        if my_match != None:
            try:
                product_id = my_match["product_id"]
            except:
                pass
        
    return product_id

In [31]:
page_links = [f"https://www.amazon.in{x}" for x in new_links]
print(page_links)
def clean_page_links(page_links=[]):
    final_page_links = []
    for url in page_links:
        product_id = extract_product_id_from_url(url)
        if product_id != None:
            final_page_links.append({"url": url, "product_id": product_id})
    return final_page_links

cleaned_links=clean_page_links(page_links)

['https://www.amazon.in/Sirona-Approved-Reusable-Menstrual-Silicone/dp/B07548J9WC/ref=zg_bs_g_boost_d_sccl_3/261-1198201-1142810?psc=1', 'https://www.amazon.in/Hykes-Compression-Support-Running-Cycling/dp/B07J6GBG8H/ref=zg_bs_g_boost_d_sccl_19/261-1198201-1142810?psc=1', 'https://www.amazon.in/prime?ref_=nav_cs_primelink_nonmember', 'https://www.amazon.in/Sirona-Disposable-Underarm-Peel-off-Unpleasant/dp/B01BKBOGDY/ref=zg_bs_g_boost_d_sccl_8/261-1198201-1142810?psc=1', 'https://www.amazon.in/now?ref=footer_amznow', 'https://www.amazon.in/FIXDERMA-Sunscreen-30-Protection-Resistant/dp/B089RJ6Z3K/ref=zg_bs_g_boost_d_sccl_5/261-1198201-1142810?psc=1', 'https://www.amazon.in/gp/browse.html?node=6648217031&ref_=nav_cs_fashion', 'https://www.amazon.in/gp/bestsellers/boost/10894277031/ref=zg_bs_nav_boost_1', 'https://www.amazon.in/gp/help/customer/display.html?nodeId=200534380&ref_=footer_privacy', 'https://www.amazon.in/hz/mycd/myx?pageType=content&ref_=nav_AccountFlyout_myk', 'https://www.am

In [32]:
cleaned_links

[{'url': 'https://www.amazon.in/Sirona-Approved-Reusable-Menstrual-Silicone/dp/B07548J9WC/ref=zg_bs_g_boost_d_sccl_3/261-1198201-1142810?psc=1',
  'product_id': 'B07548J9WC'},
 {'url': 'https://www.amazon.in/Hykes-Compression-Support-Running-Cycling/dp/B07J6GBG8H/ref=zg_bs_g_boost_d_sccl_19/261-1198201-1142810?psc=1',
  'product_id': 'B07J6GBG8H'},
 {'url': 'https://www.amazon.in/Sirona-Disposable-Underarm-Peel-off-Unpleasant/dp/B01BKBOGDY/ref=zg_bs_g_boost_d_sccl_8/261-1198201-1142810?psc=1',
  'product_id': 'B01BKBOGDY'},
 {'url': 'https://www.amazon.in/FIXDERMA-Sunscreen-30-Protection-Resistant/dp/B089RJ6Z3K/ref=zg_bs_g_boost_d_sccl_5/261-1198201-1142810?psc=1',
  'product_id': 'B089RJ6Z3K'},
 {'url': 'https://www.amazon.in/AmbiTech-PHX-01-Thermometer-Operation-Underarm/dp/B082W9S7LM/ref=zg_bs_g_boost_d_sccl_25/261-1198201-1142810?psc=1',
  'product_id': 'B082W9S7LM'},
 {'url': 'https://www.amazon.in/Vooki-Ecofriendly-Stains-Descaler-Remover/dp/B0B4GBRRR3/ref=zg_bs_g_boost_d_sccl_12

In [33]:
def perform_scrape(page_links):
    data_extracted = []
    for link in page_links:
        title, price = (None, None)
        try:
            title, price = scrape_product_page(link["url"])
        except:
            pass
        if title != None and price != None:
            print(link)
            print(title, price)
            print("------")
            product_data = {
                "title": title,
                "price": price,
                "url": link["url"],
                "product_id": link["product_id"],
            }
            data_extracted.append(product_data)
    

In [34]:
perform_scrape(page_links=cleaned_links)


{'url': 'https://www.amazon.in/Sirona-Approved-Reusable-Menstrual-Silicone/dp/B07548J9WC/ref=zg_bs_g_boost_d_sccl_3/261-1198201-1142810?psc=1', 'product_id': 'B07548J9WC'}
Sirona Reusable Menstrual Cup for Women | Medium Size with Pouch | Ultra Soft, Odour & Rash Free|100% Medical Grade Silicone|No Leakage|Protection for Up to 8-10 Hours | US FDA Registered,Pack of 1 ₹265.00
------
{'url': 'https://www.amazon.in/Hykes-Compression-Support-Running-Cycling/dp/B07J6GBG8H/ref=zg_bs_g_boost_d_sccl_19/261-1198201-1142810?psc=1', 'product_id': 'B07J6GBG8H'}
Hykes Knee cap Compression Support for Gym Running Cycling Sports Jogging Workout Pain Relief (Material: Nylon & Spandex, Color:Green, Size:Medium) ₹699.00
------
{'url': 'https://www.amazon.in/Sirona-Disposable-Underarm-Peel-off-Unpleasant/dp/B01BKBOGDY/ref=zg_bs_g_boost_d_sccl_8/261-1198201-1142810?psc=1', 'product_id': 'B01BKBOGDY'}
Sirona Disposable Underarm Sweat Pads for Men and Women, 24 Pads (Pack of 1) | Peel-off | Prevents Stains,